In [193]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-df/sample_submission.csv
/kaggle/input/spaceship-df/train.csv
/kaggle/input/spaceship-df/test.csv


In [194]:
df=pd.read_csv('/kaggle/input/spaceship-df/train.csv')

In [195]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [196]:
#unnecessary columns-'VIP','RoomService','ShoppingMall','Spa','VRDeck','Name'

In [197]:
df.drop(columns=['VIP','RoomService','ShoppingMall','Spa','VRDeck','Name'],axis=1,inplace=True)

In [198]:
df.drop(columns=['FoodCourt'],axis=1,inplace=True)

In [199]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,True


In [200]:
df.isna().sum()

PassengerId      0
HomePlanet     201
CryoSleep      217
Cabin          199
Destination    182
Age            179
Transported      0
dtype: int64

In [201]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  8693 non-null   object 
 1   HomePlanet   8492 non-null   object 
 2   CryoSleep    8476 non-null   object 
 3   Cabin        8494 non-null   object 
 4   Destination  8511 non-null   object 
 5   Age          8514 non-null   float64
 6   Transported  8693 non-null   bool   
dtypes: bool(1), float64(1), object(5)
memory usage: 416.1+ KB


In [202]:
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

In [203]:
SImputer=SimpleImputer(strategy='most_frequent')

In [204]:
imputed_data=SImputer.fit_transform(df)

In [205]:
df_cleaned=pd.DataFrame(imputed_data,columns=df.columns)

In [206]:
df_cleaned.isna().sum()

PassengerId    0
HomePlanet     0
CryoSleep      0
Cabin          0
Destination    0
Age            0
Transported    0
dtype: int64

In [207]:
df_cleaned.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,True


In [208]:
df_cleaned[['Star','Planet']]=df_cleaned['Destination'].str.split('-',expand=True)

In [209]:
df_cleaned[['Deck','DecNum','DeckSide']]=df_cleaned['Cabin'].str.split('/',expand=True)

In [210]:
df_cleaned.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,Transported,Star,Planet,Deck,DecNum,DeckSide
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,TRAPPIST,1e,B,0,P
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,True,TRAPPIST,1e,F,0,S
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,False,TRAPPIST,1e,A,0,S
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,TRAPPIST,1e,A,0,S
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,True,TRAPPIST,1e,F,1,S


In [211]:
df_cleaned['Deck'].value_counts()

F    2794
G    2758
E     876
B     779
C     747
D     478
A     256
T       5
Name: Deck, dtype: int64

In [212]:
df_cleaned['DecNum'].value_counts()

734     208
82       28
86       22
19       22
56       21
       ... 
1644      1
1515      1
1639      1
1277      1
1894      1
Name: DecNum, Length: 1817, dtype: int64

In [213]:
df_cleaned['DeckSide'].value_counts()

S    4487
P    4206
Name: DeckSide, dtype: int64

In [214]:
df_cleaned['Transported']=df_cleaned['Transported'].astype(str)

In [215]:
df_cleaned['Transported']=df_cleaned['Transported'].map({'True':1,'False':0})

In [216]:
df_cleaned['CryoSleep']=df_cleaned['CryoSleep'].astype(str)

In [217]:
df_cleaned['CryoSleep']=df_cleaned['CryoSleep'].map({'True':1,'False':0})

In [218]:
df_cleaned['PassengerId']=df_cleaned['PassengerId'].astype(str)

In [219]:
df_cleaned.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,Transported,Star,Planet,Deck,DecNum,DeckSide
0,0001_01,Europa,0,B/0/P,TRAPPIST-1e,39.0,0,TRAPPIST,1e,B,0,P
1,0002_01,Earth,0,F/0/S,TRAPPIST-1e,24.0,1,TRAPPIST,1e,F,0,S
2,0003_01,Europa,0,A/0/S,TRAPPIST-1e,58.0,0,TRAPPIST,1e,A,0,S
3,0003_02,Europa,0,A/0/S,TRAPPIST-1e,33.0,0,TRAPPIST,1e,A,0,S
4,0004_01,Earth,0,F/1/S,TRAPPIST-1e,16.0,1,TRAPPIST,1e,F,1,S


In [220]:
test_df=pd.read_csv('/kaggle/input/spaceship-df/test.csv')

In [221]:
test_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [222]:
test_df.drop(columns=['FoodCourt','VIP','RoomService','ShoppingMall','Spa','VRDeck','Name'],axis=1,inplace=True)

In [223]:
imputed_test_data=SImputer.fit_transform(test_df)
df_test_cleaned=pd.DataFrame(imputed_test_data,columns=test_df.columns)

In [224]:
df_test_cleaned.isna().sum()

PassengerId    0
HomePlanet     0
CryoSleep      0
Cabin          0
Destination    0
Age            0
dtype: int64

In [225]:
df_test_cleaned[['Star','Planet']]=df_test_cleaned['Destination'].str.split('-',expand=True)
df_test_cleaned[['Deck','DecNum','DeckSide']]=df_test_cleaned['Cabin'].str.split('/',expand=True)

In [226]:
df_test_cleaned['CryoSleep']=df_test_cleaned['CryoSleep'].astype(str)
df_test_cleaned['CryoSleep']=df_test_cleaned['CryoSleep'].map({'True':1,'False':0})

In [227]:
df_cleaned.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,Transported,Star,Planet,Deck,DecNum,DeckSide
0,0001_01,Europa,0,B/0/P,TRAPPIST-1e,39.0,0,TRAPPIST,1e,B,0,P
1,0002_01,Earth,0,F/0/S,TRAPPIST-1e,24.0,1,TRAPPIST,1e,F,0,S
2,0003_01,Europa,0,A/0/S,TRAPPIST-1e,58.0,0,TRAPPIST,1e,A,0,S
3,0003_02,Europa,0,A/0/S,TRAPPIST-1e,33.0,0,TRAPPIST,1e,A,0,S
4,0004_01,Earth,0,F/1/S,TRAPPIST-1e,16.0,1,TRAPPIST,1e,F,1,S


In [228]:
df_cleaned.drop(columns=['Cabin','Destination'],axis=1,inplace=True)
df_test_cleaned.drop(columns=['Cabin','Destination'],axis=1,inplace=True)

In [229]:
PassId=df_test_cleaned['PassengerId']

In [230]:
#df_test_cleaned[['Group','GroupNum']]=df_test_cleaned['PassengerId'].str.split('_',expand=True)
#df_cleaned[['Group','GroupNum']]=df_cleaned['PassengerId'].str.split('_',expand=True)

In [231]:
df_cleaned.drop(columns=['PassengerId'],axis=1,inplace=True)
df_test_cleaned.drop(columns=['PassengerId'],axis=1,inplace=True)

In [232]:
df_cleaned.head()

,HomePlanet,CryoSleep,Age,Transported,Star,Planet,Deck,DecNum,DeckSide
0,Europa,0,39.0,0,TRAPPIST,1e,B,0,P
1,Earth,0,24.0,1,TRAPPIST,1e,F,0,S
2,Europa,0,58.0,0,TRAPPIST,1e,A,0,S
3,Europa,0,33.0,0,TRAPPIST,1e,A,0,S
4,Earth,0,16.0,1,TRAPPIST,1e,F,1,S


In [233]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   HomePlanet   8693 non-null   object
 1   CryoSleep    8693 non-null   int64 
 2   Age          8693 non-null   object
 3   Transported  8693 non-null   int64 
 4   Star         8693 non-null   object
 5   Planet       6893 non-null   object
 6   Deck         8693 non-null   object
 7   DecNum       8693 non-null   object
 8   DeckSide     8693 non-null   object
dtypes: int64(2), object(7)
memory usage: 611.4+ KB


In [234]:
df_cleaned=pd.get_dummies(df_cleaned,columns=['HomePlanet','Star','Planet','Deck','DeckSide'])

In [235]:
df_test_cleaned=pd.get_dummies(df_test_cleaned,columns=['HomePlanet','Star','Planet','Deck','DeckSide'])

In [236]:
df_cleaned.head()

,CryoSleep,Age,Transported,DecNum,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Star_55 Cancri e,Star_PSO J318.5,Star_TRAPPIST,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,DeckSide_P,DeckSide_S
0,0,39.0,0,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
1,0,24.0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
2,0,58.0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
3,0,33.0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
4,0,16.0,1,1,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1


In [250]:
df_test_cleaned.head()

,CryoSleep,Age,DecNum,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Star_55 Cancri e,Star_PSO J318.5,Star_TRAPPIST,Planet_1e,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,DeckSide_P,DeckSide_S
0,1,27.0,3,1,0,0,0,0,1,1,...,0,0,0,0,0,0,1,0,0,1
1,0,19.0,4,1,0,0,0,0,1,1,...,0,0,0,0,0,1,0,0,0,1
2,1,31.0,0,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,0,38.0,1,0,1,0,0,0,1,1,...,0,0,1,0,0,0,0,0,0,1
4,0,20.0,5,1,0,0,0,0,1,1,...,0,0,0,0,0,1,0,0,0,1


In [237]:
Y_train=df_cleaned['Transported']

In [238]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
df_test_cleaned1=pd.DataFrame(scaler.fit_transform(df_test_cleaned),columns=df_test_cleaned.columns)
df_cleaned1=pd.DataFrame(scaler.fit_transform(df_cleaned),columns=df_cleaned.columns)

In [252]:
df_cleaned1.head()

,CryoSleep,Age,Transported,DecNum,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Star_55 Cancri e,Star_PSO J318.5,Star_TRAPPIST,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,DeckSide_P,DeckSide_S
0,-0.73277,0.715553,-1.007274,-1.191744,-1.111173,1.754795,-0.503664,-0.511013,-0.317487,0.652521,...,-0.174191,3.187347,-0.30661,-0.241218,-0.334759,-0.688215,-0.68169,-0.02399,1.032865,-1.032865
1,-0.73277,-0.329408,0.992779,-1.191744,0.899950,-0.569867,-0.503664,-0.511013,-0.317487,0.652521,...,-0.174191,-0.313741,-0.30661,-0.241218,-0.334759,1.453035,-0.68169,-0.02399,-0.968181,0.968181
2,-0.73277,2.039169,-1.007274,-1.191744,-1.111173,1.754795,-0.503664,-0.511013,-0.317487,0.652521,...,5.740821,-0.313741,-0.30661,-0.241218,-0.334759,-0.688215,-0.68169,-0.02399,-0.968181,0.968181
3,-0.73277,0.297569,-1.007274,-1.191744,-1.111173,1.754795,-0.503664,-0.511013,-0.317487,0.652521,...,5.740821,-0.313741,-0.30661,-0.241218,-0.334759,-0.688215,-0.68169,-0.02399,-0.968181,0.968181
4,-0.73277,-0.886720,0.992779,-1.189769,0.899950,-0.569867,-0.503664,-0.511013,-0.317487,0.652521,...,-0.174191,-0.313741,-0.30661,-0.241218,-0.334759,1.453035,-0.68169,-0.02399,-0.968181,0.968181


**MODEL TRAINING**

In [239]:
X_train=df_cleaned1.drop('Transported',axis=1)

In [240]:
from sklearn.ensemble import RandomForestClassifier

In [241]:
rfc=RandomForestClassifier(n_estimators=100,random_state=42)

In [242]:
X_train

,CryoSleep,Age,DecNum,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Star_55 Cancri e,Star_PSO J318.5,Star_TRAPPIST,Planet_1e,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,DeckSide_P,DeckSide_S
0,-0.732770,0.715553,-1.191744,-1.111173,1.754795,-0.503664,-0.511013,-0.317487,0.652521,0.652521,...,-0.174191,3.187347,-0.30661,-0.241218,-0.334759,-0.688215,-0.681690,-0.02399,1.032865,-1.032865
1,-0.732770,-0.329408,-1.191744,0.899950,-0.569867,-0.503664,-0.511013,-0.317487,0.652521,0.652521,...,-0.174191,-0.313741,-0.30661,-0.241218,-0.334759,1.453035,-0.681690,-0.02399,-0.968181,0.968181
2,-0.732770,2.039169,-1.191744,-1.111173,1.754795,-0.503664,-0.511013,-0.317487,0.652521,0.652521,...,5.740821,-0.313741,-0.30661,-0.241218,-0.334759,-0.688215,-0.681690,-0.02399,-0.968181,0.968181
3,-0.732770,0.297569,-1.191744,-1.111173,1.754795,-0.503664,-0.511013,-0.317487,0.652521,0.652521,...,5.740821,-0.313741,-0.30661,-0.241218,-0.334759,-0.688215,-0.681690,-0.02399,-0.968181,0.968181
4,-0.732770,-0.886720,-1.189769,0.899950,-0.569867,-0.503664,-0.511013,-0.317487,0.652521,0.652521,...,-0.174191,-0.313741,-0.30661,-0.241218,-0.334759,1.453035,-0.681690,-0.02399,-0.968181,0.968181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,-0.732770,0.854881,-0.998198,-1.111173,1.754795,-0.503664,1.956897,-0.317487,-1.532519,-1.532519,...,5.740821,-0.313741,-0.30661,-0.241218,-0.334759,-0.688215,-0.681690,-0.02399,1.032865,-1.032865
8689,1.364685,-0.747392,1.768722,0.899950,-0.569867,-0.503664,-0.511013,3.149739,-1.532519,-1.532519,...,-0.174191,-0.313741,-0.30661,-0.241218,-0.334759,-0.688215,1.466943,-0.02399,-0.968181,0.968181
8690,-0.732770,-0.190079,1.770697,0.899950,-0.569867,-0.503664,-0.511013,-0.317487,0.652521,0.652521,...,-0.174191,-0.313741,-0.30661,-0.241218,-0.334759,-0.688215,1.466943,-0.02399,-0.968181,0.968181
8691,-0.732770,0.227905,0.009032,-1.111173,1.754795,-0.503664,1.956897,-0.317487,-1.532519,-1.532519,...,-0.174191,-0.313741,-0.30661,-0.241218,2.987225,-0.688215,-0.681690,-0.02399,-0.968181,0.968181


In [243]:
rfc.fit(X_train,Y_train)

RandomForestClassifier(random_state=42)

In [244]:
Y_pred=rfc.predict(df_test_cleaned1)

In [245]:
Y_pred

array([1, 1, 1, ..., 1, 0, 0])

In [247]:
submission_df1=pd.DataFrame({'PassengerId':PassId,\
                           'Transported':Y_pred})
submission_df1.to_csv('/kaggle/working/Submission1.csv',index=False)

In [253]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  4277 non-null   object
 1   Transported  4277 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 67.0+ KB


In [ ]:
submission_df['PassengerId'].nunique()

In [ ]:
from sklearn.metrics import accuracy_score

In [249]:
df_cleaned.head()

,CryoSleep,Age,Transported,DecNum,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Star_55 Cancri e,Star_PSO J318.5,Star_TRAPPIST,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,DeckSide_P,DeckSide_S
0,0,39.0,0,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
1,0,24.0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
2,0,58.0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
3,0,33.0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
4,0,16.0,1,1,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
